**Jeel**, current issues that need working

* some customers make foreign transactions, we need to identtidy them and convert that transaction to their home currency (or a standardizer) remove that customer from the country's df



# Smart Budget Planner
Data Prepared by Raphael Lu and Jeel Faldu

In [ ]:
import pandas as pd
import numpy as np
import json

import kagglehub
from kagglehub import KaggleDatasetAdapter

## Introduction

The purpose of this project is to prepare data for a Smart Budget Planner Project. Specifically, we take a kaggle dataset of international customer credit card transactions from 01 October to 30 October 2024


**Final JSON Structure**

1.  country #('UK' 'Brazil' 'Japan' 'Australia' 'Nigeria' 'Germany' 'Mexico' 'Russia' 'France' 'Canada' 'Singapore' 'USA')

  * country_aggregate-data
      * summary_stats
           * mean_total_spend
           * median_total_spend
           * std_total_spend
      
      * currency
      
      * categories #(['Restaurant' 'Entertainment' 'Grocery' 'Gas' 'Healthcare' 'Education' 'Travel' 'Retail'])
           * category (i.e. Resturant)
               * cat_mean
               * cat_median
               * cat_std
               * sub_catagories

  * users
      * CUST_00001
           * total_spending
           * categories
                * category (i.e. Restaurant)
                     * mean
                     * highest_transaction
                          * amount
                          * sub_category
                     * cat_zscore
                     * over_benchmark: bool
                     * spending_by_sub
                        * sub_cat_1 (i.e. 'fast_food')
                * ....

2. country ...



## Data Loading and Pre-processing

### Historic Currency Conversion Rates

During data exploration, it was found that multiple customers made foreign transaction. The dictionary below was obtain with the use of Claude.

**Currency Conversion Rates**  
Source: HM Revenue & Customs (HMRC) - October 2024 Monthly Exchange Rates
Retrieved via: Claude AI (Anthropic) web search on November 9, 2025
Original source: https://www.trade-tariff.service.gov.uk/exchange_rates/view/2024-10
Date Range: October 1-31, 2024

In [ ]:
conversion_to_usd = {
    'USA': 1.0,
    'UK': 0.757,        # 1 GBP = 1.32 USD → 1 USD = 0.757 GBP
    'Brazil': 0.182,    # BRL: 7.24 per GBP → 5.48 per USD
    'Japan': 0.00706,   # JPY: 187.4 per GBP → 141.8 per USD
    'Australia': 0.679, # AUD: 1.95 per GBP → 1.47 per USD
    'Nigeria': 0.00061, # NGN: 2165.7 per GBP → 1639 per USD
    'Germany': 0.842,   # EUR: 1.19 per GBP → 1.09 EUR per USD
    'Mexico': 0.0524,   # MXN: 25.21 per GBP → 19.08 per USD
    'Russia': 0.0109,   # RUB: 121.3 per GBP → 91.8 per USD
    'France': 0.842,    # EUR (same as Germany)
    'Canada': 0.736,    # CAD: 1.79 per GBP → 1.36 per USD
    'Singapore': 0.773  # SGD: 1.71 per GBP → 1.29 per USD
}

### df Loading

In [ ]:
df = kagglehub.dataset_load(KaggleDatasetAdapter.PANDAS,
                            "ismetsemedov/transactions",
                            "synthetic_fraud_data.csv")

Using Colab cache for faster access to the 'transactions' dataset.


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7483766 entries, 0 to 7483765
Data columns (total 24 columns):
 #   Column               Dtype  
---  ------               -----  
 0   transaction_id       object 
 1   customer_id          object 
 2   card_number          int64  
 3   timestamp            object 
 4   merchant_category    object 
 5   merchant_type        object 
 6   merchant             object 
 7   amount               float64
 8   currency             object 
 9   country              object 
 10  city                 object 
 11  city_size            object 
 12  card_type            object 
 13  card_present         bool   
 14  device               object 
 15  channel              object 
 16  device_fingerprint   object 
 17  ip_address           object 
 18  distance_from_home   int64  
 19  high_risk_merchant   bool   
 20  transaction_hour     int64  
 21  weekend_transaction  bool   
 22  velocity_last_hour   object 
 23  is_fraud             bool   
dty

In [ ]:
df.head(5)

,transaction_id,customer_id,card_number,timestamp,merchant_category,merchant_type,merchant,amount,currency,country,...,device,channel,device_fingerprint,ip_address,distance_from_home,high_risk_merchant,transaction_hour,weekend_transaction,velocity_last_hour,is_fraud
0,TX_a0ad2a2a,CUST_72886,6646734767813109,2024-09-30 00:00:01.034820+00:00,Restaurant,fast_food,Taco Bell,294.87,GBP,UK,...,iOS App,mobile,e8e6160445c935fd0001501e4cbac8bc,197.153.60.199,0,False,0,False,"{'num_transactions': 1197, 'total_amount': 334...",False
1,TX_3599c101,CUST_70474,376800864692727,2024-09-30 00:00:01.764464+00:00,Entertainment,gaming,Steam,3368.97,BRL,Brazil,...,Edge,web,a73043a57091e775af37f252b3a32af9,208.123.221.203,1,True,0,False,"{'num_transactions': 509, 'total_amount': 2011...",True
2,TX_a9461c6d,CUST_10715,5251909460951913,2024-09-30 00:00:02.273762+00:00,Grocery,physical,Whole Foods,102582.38,JPY,Japan,...,Firefox,web,218864e94ceaa41577d216b149722261,10.194.159.204,0,False,0,False,"{'num_transactions': 332, 'total_amount': 3916...",False
3,TX_7be21fc4,CUST_16193,376079286931183,2024-09-30 00:00:02.297466+00:00,Gas,major,Exxon,630.60,AUD,Australia,...,iOS App,mobile,70423fa3a1e74d01203cf93b51b9631d,17.230.177.225,0,False,0,False,"{'num_transactions': 764, 'total_amount': 2201...",False
4,TX_150f490b,CUST_87572,6172948052178810,2024-09-30 00:00:02.544063+00:00,Healthcare,medical,Medical Center,724949.27,NGN,Nigeria,...,Chrome,web,9880776c7b6038f2af86bd4e18a1b1a4,136.241.219.151,1,False,0,False,"{'num_transactions': 218, 'total_amount': 4827...",True


In [ ]:
print('Unique Categories:', df['merchant_category'].unique())
print()
print('Unique Merchant Types (sub catagories):', df['merchant_type'].unique())
print()
print('Unique Countries:', df['country'].unique())
print()
print('Unique Currencies:', df['currency'].unique())

Unique Categories: ['Restaurant' 'Entertainment' 'Grocery' 'Gas' 'Healthcare' 'Education'
 'Travel' 'Retail']

Unique Merchant Types (sub catagories): ['fast_food' 'gaming' 'physical' 'major' 'medical' 'online' 'hotels'
 'pharmacy' 'premium' 'events' 'supplies' 'airlines' 'local' 'booking'
 'streaming' 'transport' 'casual']

Unique Countries: ['UK' 'Brazil' 'Japan' 'Australia' 'Nigeria' 'Germany' 'Mexico' 'Russia'
 'France' 'Canada' 'Singapore' 'USA']

Unique Currencies: ['GBP' 'BRL' 'JPY' 'AUD' 'NGN' 'EUR' 'MXN' 'RUB' 'CAD' 'SGD' 'USD']


### Drop Irrelevant Columns

In [ ]:
# Drop Fraud Cases
df = df[df['is_fraud'] == False]

# Cols to drop
drop_cols = ['card_number', 'city_size', 'card_type', 'card_present',
             'device_fingerprint', 'ip_address',
             'high_risk_merchant','velocity_last_hour','device','is_fraud',
             'transaction_hour','city']

### Create Transactions df

In [ ]:
df_transactions = df.copy().drop(drop_cols, axis=1)
df_transactions

,transaction_id,customer_id,timestamp,merchant_category,merchant_type,merchant,amount,currency,country,channel,distance_from_home,weekend_transaction
0,TX_a0ad2a2a,CUST_72886,2024-09-30 00:00:01.034820+00:00,Restaurant,fast_food,Taco Bell,294.87,GBP,UK,mobile,0,False
2,TX_a9461c6d,CUST_10715,2024-09-30 00:00:02.273762+00:00,Grocery,physical,Whole Foods,102582.38,JPY,Japan,web,0,False
3,TX_7be21fc4,CUST_16193,2024-09-30 00:00:02.297466+00:00,Gas,major,Exxon,630.60,AUD,Australia,mobile,0,False
6,TX_e0d7eb37,CUST_89147,2024-09-30 00:00:03.149440+00:00,Grocery,online,Instacart,2606.19,BRL,Brazil,mobile,0,False
7,TX_eb55c2be,CUST_10150,2024-09-30 00:00:06.295911+00:00,Travel,hotels,Westin,828.33,EUR,Germany,web,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...
7483761,TX_58a762fa,CUST_11344,2024-10-30 23:59:58.926575+00:00,Retail,online,AliExpress,887.32,SGD,Singapore,web,0,False
7483762,TX_dabcf671,CUST_38961,2024-10-30 23:59:58.950801+00:00,Grocery,physical,Kroger,356.06,EUR,France,web,0,False
7483763,TX_ab567eb4,CUST_75576,2024-10-30 23:59:58.972155+00:00,Grocery,physical,Kroger,391.96,SGD,Singapore,web,0,False
7483764,TX_5ae61846,CUST_82508,2024-10-30 23:59:58.996608+00:00,Retail,online,eBay,601.71,USD,USA,web,0,False


### Timestamp

In [ ]:
df_transactions['timestamp'] = pd.to_datetime(df['timestamp'], format='mixed')

In [ ]:
df_transactions['timestamp'].min(), df_transactions['timestamp'].max()

(Timestamp('2024-09-30 00:00:01.034820+0000', tz='UTC'),
 Timestamp('2024-10-30 23:59:59.101885+0000', tz='UTC'))

In [ ]:
df_transactions['date_transact'] = df_transactions['timestamp'].dt.strftime('%d-%m-%Y')

In [ ]:
df_transactions = df_transactions[df_transactions['date_transact'] != 9]
df_transactions.head()

,transaction_id,customer_id,timestamp,merchant_category,merchant_type,merchant,amount,currency,country,channel,distance_from_home,weekend_transaction,date_transact
0,TX_a0ad2a2a,CUST_72886,2024-09-30 00:00:01.034820+00:00,Restaurant,fast_food,Taco Bell,294.87,GBP,UK,mobile,0,False,30-09-2024
2,TX_a9461c6d,CUST_10715,2024-09-30 00:00:02.273762+00:00,Grocery,physical,Whole Foods,102582.38,JPY,Japan,web,0,False,30-09-2024
3,TX_7be21fc4,CUST_16193,2024-09-30 00:00:02.297466+00:00,Gas,major,Exxon,630.60,AUD,Australia,mobile,0,False,30-09-2024
6,TX_e0d7eb37,CUST_89147,2024-09-30 00:00:03.149440+00:00,Grocery,online,Instacart,2606.19,BRL,Brazil,mobile,0,False,30-09-2024
7,TX_eb55c2be,CUST_10150,2024-09-30 00:00:06.295911+00:00,Travel,hotels,Westin,828.33,EUR,Germany,web,0,False,30-09-2024


### Categories and Sub-categories

In [ ]:
for cat in df_transactions['merchant_category'].unique():
  mask = df_transactions['merchant_category'] == cat
  print(f'Sub Categories in {cat}', df_transactions[mask]['merchant_type'].unique())
  print()

Sub Categories in Restaurant ['fast_food' 'premium' 'casual']

Sub Categories in Grocery ['physical' 'online']

Sub Categories in Gas ['major' 'local']

Sub Categories in Travel ['hotels' 'airlines' 'booking' 'transport']

Sub Categories in Healthcare ['medical' 'pharmacy']

Sub Categories in Entertainment ['events' 'gaming' 'streaming']

Sub Categories in Education ['supplies' 'online']

Sub Categories in Retail ['physical' 'online']



### Handeling Foreign Transactions

In [ ]:
# creates dict (cust_id : homecountry)
home_country_map = df_transactions[df_transactions['distance_from_home'] == 0].groupby('customer_id')['country'].first()

# .map calls on the dict
df_transactions['home_country'] = df_transactions['customer_id'].map(home_country_map)

df_transactions['sales_loc_rate_to_usd'] = df_transactions['country'].map(conversion_to_usd)
df_transactions['home_rate_to_usd'] = df_transactions['home_country'].map(conversion_to_usd)

foreign_mask = df_transactions['distance_from_home'] == 1

df_transactions.loc[foreign_mask, 'amount'] =  (df_transactions.loc[foreign_mask, 'amount'] * df_transactions.loc[foreign_mask, 'sales_loc_rate_to_usd'] / df_transactions.loc[foreign_mask, 'home_rate_to_usd'])

df_transactions.loc[foreign_mask, 'country'] = df_transactions.loc[foreign_mask, 'home_country']

df_transactions = df_transactions.drop(['sales_loc_rate_to_usd', 'home_rate_to_usd', 'home_country'], axis=1)

### Country dfs (dict)

In [ ]:
county_df_dict = {}

for country in df_transactions['country'].unique():
  county_df_dict[country] = df_transactions[df_transactions['country'] == country]

In [ ]:
print(county_df_dict.keys())

dict_keys(['UK', 'Japan', 'Australia', 'Brazil', 'Germany', 'Nigeria', 'Russia', 'France', 'Canada', 'Singapore', 'USA', 'Mexico'])


In [ ]:
home_country_map_2 = df_transactions[df_transactions['distance_from_home'] == 0].groupby('customer_id')['country']
df_multinat = home_country_map_2.nunique()

df_multinat[df_multinat >= 2].max


<bound method Series.max of customer_id
CUST_11396    2
CUST_11430    2
CUST_12255    2
CUST_13670    2
CUST_14192    2
             ..
CUST_97591    2
CUST_98389    2
CUST_98661    2
CUST_98694    2
CUST_99140    2
Name: country, Length: 118, dtype: int64>

In [ ]:
duplicate_users = df_transactions.groupby('customer_id')['country'].nunique()
assert (duplicate_users == 1).all(), f"Found {(duplicate_users > 1).sum()} users in multiple countries!"
print("✓ Validation passed: Each user is in exactly one country")

AssertionError: Found 118 users in multiple countries!

------
# JEEL PLEASE WORK BELOW THIS LINE
-----------